<a href="https://colab.research.google.com/github/Daviunb6202/Projetos-CIS-UnB/blob/main/Projeto_6%20Inicial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
 from google.colab import drive

In [ ]:
!pip install --user tensorflow-addons

In [ ]:
!pip install --user tensorflow --upgrade --force-reinstall

In [ ]:
pip install --upgrade tensorflow-gpu

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_md

In [ ]:
!pip install tensorflow_transform

In [91]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras 
import matplotlib.pyplot as plt
import sklearn as sk  
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

## Importação

In [95]:
df = pd.read_csv("/content/drive/MyDrive/Musical_instruments_reviews.csv")
df.dropna(axis=0,inplace=True)
df.reset_index(inplace=True)
df.isna().sum()

index             0
reviewerID        0
asin              0
reviewerName      0
helpful           0
reviewText        0
overall           0
summary           0
unixReviewTime    0
reviewTime        0
dtype: int64

## Pré-processamento 

In [96]:
df['overall'] = pd.Categorical(df['overall'])
df['overall'] = df.overall.cat.codes
df['overall'] = [1 if text>5 else 0 for text in df['overall'] ] #Aqui achei que fez mais sentido separar o texto em bom ou ruim,
                                                               #pelo menos em um primeiro momento 
target = df.pop('overall')
df = df['reviewText']

## Segundo problema, não sei exatamente como codar essa dimensão dos time steps ! 

In [115]:
train_size = round(len(df)*0.7)
valid_size = round(len(df)*0.15)

X_train, X_valid, X_test = df[:train_size], df[train_size:valid_size], df[valid_size:]
y_train, y_valid, y_test = target[:train_size], target[train_size:valid_size], target[valid_size:]

## Problema de Dataset.from_tensor_slices() por meio de tuplas e não consegue rodar em outras funções 

In [53]:
df = tf.convert_to_tensor(df, dtype=tf.string)    #aplicar para train, valid e test quando desse certo ?
target = tf.convert_to_tensor(target, dtype=tf.int32)
dataset = tf.data.Dataset.from_tensor_slices((df, target))         

In [54]:
for df, target in dataset.take(1):
  print(df, target)

tf.Tensor(b"Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing,", shape=(), dtype=string) tf.Tensor(0, shape=(), dtype=int32)


In [ ]:
vocab_size = 10000
tk = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size)
tk.fit_on_texts(dataset)

tk.sequences_to_matrix(tk.texts_to_sequences(dataset), mode='tfidf')

In [ ]:
from collections import Counter
vocabulary = Counter()
for review in dataset:
  vocabulary.update(list(review.numpy()))

In [22]:
truncated_vocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]

In [ ]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

## Outro caminho de préprocessamento por meio do sklearn 

In [109]:
nltk.download("wordnet")
nltk.download("stopwords")
nltk.download("punkt")


from nltk import word_tokenize
from nltk.corpus import stopwords
import re
def Prep(text):
  text.lower()         #normaliza tudo para minúsculas
  text = re.sub(r"[^\w\s]", "", text) #padrão ReGeX em que seja tudo menos(^) palavras(\w) e espaços (\s)
  stemmer = PorterStemmer()
  words = word_tokenize(text)  #Finalmente quebra os termos em tokens 
  stemmed_words = []
  en_stopwords =  stopwords.words('english') #importando stopwords(artigos,interjeições,preposições)
  for word in words:
    if word not in en_stopwords:          
      stemmed_words.append(stemmer.stem(word))       #se não for é adicionado 
  return  " ".join(str(np.array(stemmed_words)).split()) #Une tudo por espaços 


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [110]:
for i in range(len(df)):
    result = Prep(df[i])
    df[i] = result


## Outro erro, o processamento de word embedding importado não funciona mais

In [ ]:
import spacy 

nlp = spacy.load('en_core_web_md')

docs = [nlp(text) for text in X_train]   #Adapta o sistema de word embedding importado para o dataset em questão com diferentes probabilidades para cada palavra
print(docs[0].vector)

In [111]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer #Une o Tfidf com o a função CountVectorizer (Bag of words)
                                                   #fazendo então a vetorização com um processo de balancear as ocorrências das palavras(e por conseguinte os pesos)
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df)
print(vectorizer.get_feature_names())

['000', '00045', '001', '00115', '0012', '0012053', '0013', '0022uf', '0028', '0032', '0034', '004', '0054in', '007usb', '008', '009', '009042', '009042or', '01', '010', '01082014', '011', '012', '0125', '012dlx', '012the', '013', '0135013', '013i', '016', '016ni', '017', '017i', '02', '024w', '025', '025026', '02534', '028', '0290', '0305', '0313', '0318', '032', '03202013thi', '032w', '034036', '0375', '038', '0390', '042', '042w', '046mm', '048', '05', '05034', '050mm', '053', '053whighli', '055', '05all', '05mm', '060', '060mm', '06mm', '07', '070mm', '07142013', '0733150406', '07etc', '08', '0818', '083', '085', '088mm', '08nov2010', '09', '0942', '099', '0990502000', '0_o', '0db', '10', '100', '1000', '10000', '1000000x', '10002000hz', '1000500', '1000ma', '1000maoutput', '1000page', '1000x', '100250', '1003', '10034', '100500hz', '100con', '100digitech', '100hz', '100i', '100joyo', '100low', '100ma', '100maoutput', '100mavisu', '100mm', '100pound', '100pro', '100th', '100w', '10

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Repetindo train test split pro vetor

In [121]:
train_size = round(len(df)*0.7)
valid_size = round(len(df)*0.15)

X_train, X_valid, X_test = vectors[:train_size], vectors[train_size:valid_size], vectors[valid_size:]
y_train, y_valid, y_test = target[:train_size], target[train_size:valid_size], target[valid_size:]

The first layer is an Embedding layer, which will convert word IDs into embeddings
(introduced in Chapter 13). The embedding matrix needs to have one row per word
ID (vocab_size + num_oov_buckets) and one column per embedding dimension
(this example uses 128 dimensions, but this is a hyperparameter you could tune).
Whereas the inputs of the model will be 2D tensors of shape [batch size, time steps],
the output of the Embedding layer will be a 3D tensor of shape [batch size, time steps,
embedding size].

## Modelo Simples RNN 


In [113]:
embed_size = 128
vocab_size = 10000 
num_oov_buckets = 1000 
model = keras.models.Sequential([
keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size, input_shape=[None]),
keras.layers.SimpleRNN(32),
keras.layers.Dense(10, activation='relu'),
keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x=X_train, y = y_train, epochs=20,validation_data=(X_valid, y_valid))
#model.save('Modelo_melhorado.h5')

## LTSM Unidirecional 

In [280]:
model = keras.models.Sequential([
keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
keras.layers.LSTM(20, return_sequences=True),
keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x=X_train, y = y_train, epochs=20,validation_data=(X_valid, y_valid))
#model.save('Modelo_melhorado.h5')

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 5) 
plt.show()

# LTSM Bidirecional 


In [138]:
model = keras.models.Sequential([
keras.layers.Bidirectional(keras.layers.GRU(10, return_sequences=True)),
keras.layers.Bidirectional(keras.layers.GRU(10, return_sequences=True)),
keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x=X_train, y = y_train, epochs=20,validation_data=(X_valid, y_valid), steps_per_epoch=30 ,validation_steps=30)
#model.save('Modelo_melhorado.h5')

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 5) 
plt.show()

## Meios de comparação no sklearn

In [ ]:
from sklearn import svm
clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(vectors, y_train)

In [ ]:
from sklearn import svm

X_train_word_vector = [x.vector for x in docs]
clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(X_train_word_vector, y_train)

## Tentativa tensorflow dataset 

In [ ]:

def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = tf.squeeze(tf.constant([dataframe.pop('overall')]), axis=0)
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels)).batch(
        batch_size)
  return ds.batch(batch_size).shuffle(len(dataframe)).prefetch(1)

ds = df_to_dataset(df)

#target = df['overall'].copy()
#df = df.drop(labels = 'overall', axis=1)
#df = np.asarray(df).astype(np.float32)
#target =  np.asarray(target).astype(np.float32)
#dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))
#X_train.shuffle(400)

In [ ]:
#train_size = int(len(ds))
full_ds_size = len(ds)
train_ds_size = int(0.70 * full_ds_size)
valid_ds_size = int(0.15 * full_ds_size)

train_ds = ds.take(train_ds_size)
remaining = ds.skip(train_ds_size)  
valid_ds = remaining.take(valid_ds_size)
test_ds = remaining.skip(valid_ds_size)

In [ ]:
def preprocess(X_batch, y_batch):
  X_batch = tf.strings.substr(X_batch, 0, 300)
  X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
  X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
  X_batch = tf.strings.split(X_batch)
  return X_batch.to_tensor(default_value=b"<pad>"), y_batch